**Introduction:**

This notebook is really an addendum to my previous notebook (Getting Started with RTKLIB).  It includes the additional steps required to run GNSS PPK solutions for the raw data sets using rtklib-py, a python version of RTKLIB, instead of the original C version.  Like the previous notebook, it is not intended to be run here, but downloaded to a local computer and run there.  I ran this code in Windows, but with minor modificatons it should run in Linux as well.

This code is primarily intended for those planning to develop or modify alogrithms internal to the PPK solutions and not just running the code as-is or with just configuration changes. For those users, the C code will run much faster.  However, the python version provides a friendlier development platform.  When development is complete, the modified python code can either be run on the complete data set on a faster PC with a little patience, or the completed changes can be fairly easily ported back into the C code since two code sets are very closely aligned.  This alignment includes file names, function names, variable names, and comments.  The code does not align on a line by line basis because of extensive use of Numpy in the python code, but structurally it is very similar.

When following these intructions, it will be easier if you use the folder structure as described in the previous notebook.


**Step 1: Retrieve base observation and satellite navigation files**

This step is identical as step 1 for the RTKLIB code, so please follow the directions in the previous notebook for this step.

**Step 2: Converting base observation files from rinex2 to rinex3 format**

Rtklib-py is only a subset of RTKLIB and not all features are supported.  We will need to use RTKLIB to convert the base station observations from rinex2 format which rtklib-py does not support, to rinex3 format, which it does.

The following code will do that.  In addition, this code filters out all the signal types in the original files that are not necessary for this exercise.

Before running the code you will need to retrieve the RTKLIB convbin executable and place it into the GSDC_2022/rtklib folder.  Follow the instructions in the previous notebook to do this.

In [ ]:
""" 
rinex2_to_rinex3.py - Convert rinex 2 obs files to rinex 3
"""

import os
from os.path import join
import subprocess

# specify input folder and files
datadir = r'C:\gps\GSDC_2022\data\test'
obsfiles = '*0.2*o' # base obs files with wild cards
sigmask = 'GL1C,GL5X,RL1C,EL1X,EL5X'

# Setup for RTKLIB 
binpath  = r'C:\gps\GSDC_2022\rtklib\convbin'


# get list of data sets in data path
datasets = os.listdir(datadir)
    
for dataset in datasets:
    try:
        os.chdir(join(datadir, dataset))
    except:
        continue
    print(dataset)
    rtkcmd = '%s %s -r rinex -mask %s -od -os -o base.obs' % \
        (binpath, obsfiles, sigmask)
    subprocess.run(rtkcmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

**Step 3: Download the android-rinex and rklib-py libraries and create rtklib-py configuration file:**

Download the android-rinex and rtklib-py libraries using the code below.  Place them into the GSDC_2022/python/android-rinex folder and the GSDC_2022/python/rtklib-py folder.

You will also need a configuration file for the solution. Copy the config file below to the GSDC_2022/config folder with a file name of ppk_phone_0510.py

In [ ]:
!git clone https://github.com/rtklibexplorer/android_rinex.git
!git clone https://github.com/rtklibexplorer/rtklib-py.git

In [ ]:
# ppk_phone_0510.py - config file for rtklib-py PPK solution

from rtkcmn import uGNSS, rSIG

# ----------- PPK options -------------------------------
nf = 2                   # num frequencies ( 1 or 2)
pmode = 'kinematic'      # static, kinematic
filtertype = 'combined_noreset'  # forward, backward, combined, combined_noreset
use_sing_pos = False     # run initial single precision sol each epoch, not
                         # necessary unless receiever clock errors are large
elmin = 15               # minimum elevation for float solution (degrees)
cnr_min = 24             # min signal strength (dB-Hz)
excsats = []             # excluded sats

maxinno = 1              # outlier threshold for phase (m)
maxage = 30              # mag age of differential
maxout = 4               # maximum outage [epoch]
thresdop = 5             # cycle slip detection by doppler method
thresslip = 0.10         # cycle slip detection by geom-free LC
interp_base = False      # interpolate base observations

# ------------  Kalman Filter Statistics ------------------------
eratio = [300, 100]    # ratio between psuedorange noise and carrier phase noise for L1, L5
efact = {uGNSS.GPS: 1.0, uGNSS.GLO: 1.5, uGNSS.GAL: 1.0} # relative weighting of each constellation
err = [0, 0.003, 0.003, 0.0, 0, 5e-12]  # err sigmas [-, base, el, rcvstd, bl, satclk]

accelh = 3               # horiz accel noise sigma (m/sec2)
accelv = 1               # vert accel noise sigma (m/sec2)
prnbias = 1e-2           # Carrier phase bias sigma ( cycles)
sig_p0 = 30.0            # initial pos sigma (m)
sig_v0 = 10.0            # initial vel/acc sigma (m/sec)
sig_n0 = 30.0            # inital bias sigma (m)

#  -------------Ambiguity resolution options ----------------
armode = 0               # 0:off, 1:continuos,3:fix-and-hold
thresar1 = 0.05           # max pos variation for AR (and accel update in kalman filter)
# the rest of these aren't used since AR is disabled
minlock = 0              # min consecutive fix samples to include sat in AR 
thresar = 3              # AR threshold
glo_hwbias = 0.0         # GLONASS HW bias
elmaskar = 15            # elevation mask for AR
var_holdamb = 0.1        # Hold ambiguity variance (m)
minfix = 20              # min fix samples to set hold
minfixsats = 4           # min sat pairs to test for fix
minholdsats = 5          # min sat pairs to test for hold
mindropsats = 10         # min sat pairs to drop sats from AR

# -----------  Single precision parameters ----------------------------
sing_p0 = 100            # initial pos sigma
sing_v0 = 10             # initial vel/acc sigma
sing_elmin = 10          # minimum elevation (degrees)

# -------------  Base and Rover positions ------------------
# base position, set to zeros to use rinex header pos
#rb = [-2703115.9211, -4291767.2078, 3854247.9065] # SLAC
rb = [0, 0, 0]  # will set based on base file name

# initial rover position/velocity for alignment to RTKLIB
#rr_f = [-2694556.682853,  -4296492.691977,   3854819.048563, # forwards
#        -0.009033 ,       0.042808,         -0.027949] 
#rr_b = [-2709836.020965, -4269097.509128, 3874376.664473, # backwards
#      0.018097,      0.051379,     -0.027851 ]
# Set to zero to use standard precision computed starting position
rr_f = rr_b  = [0, 0, 0]


# ----------- Configure observation signals ----------------
gnss_t = [uGNSS.GPS, uGNSS.GLO, uGNSS.GAL]

sig_tbl = {'1C': rSIG.L1C, '1X': rSIG.L1X, '1W': rSIG.L1W,
           '2W': rSIG.L2W, '2L': rSIG.L2L, '2X': rSIG.L2X,
           '5Q': rSIG.L5Q, '5X': rSIG.L5X, '7Q': rSIG.L7Q,
           '7X': rSIG.L7X}
skip_sig_tbl = {uGNSS.GPS: [],   # skip these obs
                uGNSS.GLO: [],
                uGNSS.GAL: [],
                uGNSS.QZS: []}

# set these from table below
freq_ix0 = {uGNSS.GPS: 0, uGNSS.GLO: 4, uGNSS.GAL: 0} # L1
freq_ix1 = {uGNSS.GPS: 2, uGNSS.GLO: 5, uGNSS.GAL: 2} # L5

# ---------- Frequencies currently supported-------------
freq = [1.57542e9,   # L1/E1
        1.22760e9,   # L2
        1.17645e9,   # L5/E5a/B2a
        1.20714e9,   # E5b
        1.60200E9,   # G1
        1.24600E9]   # G2
dfreq_glo = [0.56250E6, 0.43750E6]




**Step 4: Convert raw files and run PPK solutions**

Follow the instructions and use the code from step 4 in the previous notebook after replacing the following set of parameter definitions in the previous code.  This will run the python version of RTKLIB instead of the C version and to use the rinex3 base files instead of the rinex2 versions.

In the main code at the bottom of the file, the execution can be set up as either sequential or multiprocessing by commenting or uncommenting the appropriate lines, both for the file conversion, and for running the solutions. It is easier to debug when run sequentially but is much slower. I recommend running each step sequentially until you are convinced it's working, then switch it to multiprocessing.  

However, there appears to be a path issue when running in multiprocessing mode so for now you will either need to set your PYTHONPATH to include the android_rinex/src and rtklib-py/src folders or move the files in those folders to the python folder.

In [ ]:
# Set solution choices
ENABLE_PY = True        # Use RTKLIB-PY to generate solutions 
ENABLE_RTKLIB = False   # Use RTKLIB to generate solutions

#basefiles = '../*0.2*o' # rinex2, use this for rtklib only
basefiles = '../base.obs' # rinex3, use this for python only

**Step 5: Combine PPK solutions into a single .csv file**

Follow the instructions for step 5 in the previous notebook after replacing the following set of parameter definitions in the previous code.

In [ ]:
########### Input parameters ###############################

DATA_SET = 'test'
SOL_TAG = '_py0510'
datapath = r'C:\gps\GSDC_2022\data'
hdrlen = 1    # 25 for RTKLIB, 1 for rtklib-py

**Step 6: Filtering out RTKLIB solutions with hardware clock discontinuites**

Follow the instructions for step 6 in the previous notebook

**Step 7: Submit csv file to Kaggle**

You can now submit the modified csv file to Kaggle. I have not done this yet, but based on the results from the training set and from last year's results, I would expect the score to be just slightly better than the RTKLIB results.

See the previous notebook for final thoughts and suggestions on improving the results.